# Objectifs

* Calculer la dispersion d'un nuage de point: comprendre la différence entre l'inertie interclasse et l'inertie intraclasse
* Définir l'algorithme kmeans
* Connaître les étapes d'implémentation 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation
import time

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

from kmeans.algorithm import assign_cluster,compute_dispersion
from kmeans.distances import euclidian_dist

In [ ]:
FIGSIZE = (10,8)
VERBOSE = True
%matplotlib inline

# Définitions 

## Objectif de l'algorithme

L'algorithme de K-means est une méthode d'apprentissage non-supervisée dont l'objectif est de regrouper les observations qui sont similaires en ayant des groupes qui sont différents entre eux de façon significative.

## Similitude

La similarité de deux observations est mesurée par la distance qui les sépare. Il existe plusieurs possibilités pour définir une distance, par exemple:

* Distance euclidienne: 
    $$||X - Y||_2 = \sqrt{\sum_i(X_i - Y_i)^2}$$
    
* Distance euclidienne pondérée: 
    $$||X - Y|| = \sqrt{ \sum_i w_i \times(X_i - Y_i)^2}$$
    
* Distance de Manhattan:
    $$||X - Y|| = \sum_i|X_i - Y_i|$$

Dans le cas de la méthode de K-means que nous allons voir, la distance utilisée est **la distance euclidienne**.

La dispersion d'un nuage de point est alors évaluée par la variance:
$$ \frac{1}{n} \sum_{i=1}^{n} ||X_{i} - \mu||_2^2 $$

avec $\mu = \frac{1}{n}\sum_{i=1}^n X_{i}$ le centre de gravité du nuage.

Dans le cas d'un nuage de point constitué de $K$ classes, la dispersion devient:

$$ \underbrace{\sum_{k=1}^{K} \left(\frac{1}{n_k} \sum_{i=1}^{n_k} ||X_{i}^{(k)} - \mu_k||_2^2 \right)}_{inertie~intraclasse} +   \underbrace{\sum_{k=1}^{K} \frac{n_k}{n} ||\mu_{k} - \mu||_2^2}_{inertie~interclasse} $$

### Notation

* $X_i$ l'observation $i$ de dimension P: $X_i = (X_{i1},\dots,X_{iP})$
* $X_{i}^{(k)}$ l'observation $i$ du cluster $k$: $X_{i}^{(k)} = (X_{i1}^{(k)},\dots,X_{iP}^{(k)})$

### Définition

* l'inertie intraclasse représente la concentration/homogénéité au sein de chaque classe. Plus elle est faible, plus les éléments des classes sont proches entre elle.

* L'inertie interclasse représente la dispersion d'une classe à l'autre.

Une classification satisfaisante est un regroupement avec une faible inertie intraclasse et une forte inertie interclasse.


# Dispersion

##  d'un nuage de points

**Question:** Calculez la dispersion du nuage de points ci-dessous:

In [ ]:
data = np.array(
    [
        [0.5,-2.5],
        [1.5,4],
        [1,0]
    ]
)

**valeur à trouver:**

In [ ]:
data=pd.DataFrame(data)
cluster = np.array([1,1,1])
data["cluster"] = cluster
cluster_centroids = data.groupby(cluster).agg("mean").drop(columns = "cluster")
compute_dispersion(data.drop(columns = "cluster"), cluster_centroids)

In [ ]:
cluster_centroids

On peut utiliser le code suivant pour un nuage plus compliqué:

In [ ]:
data = np.array(
    [[0.5,1.3],
     [0.75,3],
     [1,2],
     [0.5,2.25],
     [2,1],
     [1.75,1.5],
     [1.6,2.75]]
)

mu_1 = np.mean(data[:,0])
mu_2 = np.mean(data[:,1])

mu = np.array([mu_1,mu_2])
if VERBOSE:
    plt.scatter(x = data[:,0],
            y = data[:,1],
            color = "red")

    plt.scatter(mu[0],mu[1])
    plt.show()

In [ ]:
distance_to_mean = np.sqrt(np.sum((data - mu)**2,axis=1))
dispersion = np.mean(distance_to_mean**2)

if VERBOSE:
    print(dispersion)

## de deux clusters de points

In [ ]:
clusters = np.array([1,0,1,1,1,1,0])
data = pd.DataFrame(data)
data["cluster"] = clusters
cluster_centroids = data.groupby("cluster").agg("mean")

In [ ]:
cluster_centroids

In [ ]:
data

In [ ]:
dispersion_clusters = compute_dispersion(data.drop(columns = "cluster"), cluster_centroids)
if VERBOSE:
    print(dispersion_clusters)

Idées de TD: 
* Prouver que la somme des inerties intra clusters est toujours inférieure à l'inertie totale
* Prouver que le point qui minimise la somme des distances au carré aux points du nuage est la moyenne $\mu$. C'est grâce à ce résultat que la moyenne sert de "représentant" de cluster 

# Problème à résoudre

On a N observations avec P caractéristiques. Mais le nombre d'observations est trop élevé pour bien comprendre le comportement des individus. Nous allons donc réduire le nombre d'observations de N à n (n<<N).

*Donner la relation entre variance inter cluster et intra cluster*

# Implémentation

Dans cette section, nous allons voir dans le détail comment l'algorithme du kmeans est implémenté

Si on suppose K clusters,
1. choisir K observations parmi les observations pour servir de centroïdes initiaux
2. affecter chaque observation au centroïde dont elle est le plus proche
3. recalculer les centroïdes de chaque cluster
4. répéter les opérations 2 et 3 jusqu'à convergence

In [ ]:
BUCKET_SIZE = 40
NUMBER_BUCKET = 3

In [ ]:
X = np.concatenate(
    (
        np.random.normal(0,4,BUCKET_SIZE) ,
        np.random.normal(-7.5,3,BUCKET_SIZE),
        np.random.normal(3,3,BUCKET_SIZE)
    ),
    axis = None
)

Y = np.concatenate(
    (
        np.random.normal(2,1.5,BUCKET_SIZE), 
        np.random.normal(-5,1.5,BUCKET_SIZE),
        np.random.normal(-5,3,BUCKET_SIZE)
    ), 
    axis = None
)

true_label = np.repeat(
    [0,1,2],
    [BUCKET_SIZE] * 3
)

dataset = pd.DataFrame(
    {
        "X" : X,
        "Y" : Y
    }
)

In [ ]:
if VERBOSE:
    plt.figure(figsize=FIGSIZE)
    plt.scatter(X,Y,c=true_label)
    plt.show()

In [ ]:
chosen_index = np.random.randint(NUMBER_BUCKET * BUCKET_SIZE, size=NUMBER_BUCKET)
centroides = dataset.iloc[chosen_index]
iteration = 0

while iteration < 10:
    cluster,errors = assign_cluster(dataset, centroides)
    dataset["cluster"] = cluster
    centroides = dataset.groupby('cluster').agg('mean').loc[:,['X','Y']]
    print(centroides)
    iteration += 1

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
chosen_index = np.random.randint(NUMBER_BUCKET * BUCKET_SIZE, size=NUMBER_BUCKET)
centroides = dataset.iloc[chosen_index]

cluster,errors = assign_cluster(dataset, centroides)
dataset["cluster"] = cluster

colors = sns.color_palette()
fig = plt.figure(figsize=(10, 8))
ax = plt.axes()


def init():
    return [fig]


def animate(i):
    global centroides
    global dataset
    centroides = dataset.groupby('cluster').agg('mean').loc[:,['X','Y']]
    cluster,errors = assign_cluster(dataset, centroides)
    dataset["cluster"] = cluster
    ax.cla()
    ax.scatter(dataset.X, dataset.Y,c=dataset.cluster)
    ax.scatter(centroides.X,centroides.Y,c="red")
    ax.set_xticklabels("")
    ax.set_yticklabels("")
    time.sleep(0.5)
    return [fig]


ani = animation.FuncAnimation(
    fig, animate, init_func=init, frames=40, interval=200, blit=True
)
plt.show()

# Discussion

1. Quelles devraient être les conditions d'arrêt de l'algorithme?

# **FIN DU NOTEBOOK**